In [ ]:
# Eksempel på hvordan man kan søge og analysere data i GC2

Denne Notebook viser hvordan man vha. SQL kan søge i en GC2 database. Eksemplet anvender gc2.io databasen med offentlige tilgængelige data. Eksemplet er en simple søgning på en samlet ejendom vha. adresse og viser hvordan ejendommen kan anvendes til at søge videre i DKJord dataene for at se, om der er registreret forurening på ejendommen.

In [ ]:
Først hentes gc2-python-client Python Package Index.

In [ ]:
!pip install --index-url https://test.pypi.org/simple/ gc2-python-client

In [ ]:
Importer moduler. Udover GC2 vil vi anvende Pandas og Folium, som hhv. bruges til datatabeller og kort.

In [ ]:
import gc2
import folium
import pandas

In [ ]:
Vi sætter en forbindelse op til dk.gc2.dk med en test-bruger.

In [ ]:
dk = gc2.Gc2("https://dk.gc2.io")
dk.set_authentication(user="dk_test", pw="Pwd1234!", db="dk")

In [ ]:
Vi gør to SQL objekter klar, så vi kan hente ejendoms-data og dkjord-data.

In [ ]:
ejendom = gc2.Sql(dk)
dkjord = gc2.Sql(dk)

In [ ]:
Så kan vi lave en forespørgsel på en samlet ejendom ud fra en adresse.

In [ ]:
husnr = "34"
vejkode = "0295"
postnr = "5853"

ejendom.run(f"""
SELECT sfe_ejendomsnummer,ST_Multi(ST_Union(the_geom)), ST_astext(ST_Multi(ST_Union(the_geom))) as wkt FROM matrikel.jordstykke
    WHERE sfe_ejendomsnummer = (SELECT sfe_ejendomsnummer FROM matrikel.jordstykke
    WHERE (the_geom && (SELECT ST_transform(the_geom, 25832) FROM dar.adgangsadresser WHERE husnr='{husnr}' AND vejkode='{vejkode}' AND postnr='{postnr}')) AND ST_Intersects(the_geom, (SELECT ST_transform(the_geom, 25832) FROM dar.adgangsadresser WHERE husnr='{husnr}' AND vejkode='{vejkode}' AND postnr='{postnr}'))) group by sfe_ejendomsnummer
""")
pandas.DataFrame(ejendom.data, columns=ejendom.columns)

In [ ]:
Derefter kan vi bruge ejendommens geometri til at finde evt. forurenet jord.

In [ ]:
dkjord.run(f"""
SELECT * from dkjord.dkjord_v1 where ST_intersects(the_geom, ST_geomfromtext('{ejendom.data[0][1]}',25832))
""")
pandas.DataFrame(dkjord.data, columns=dkjord.columns)

In [ ]:
Vi sætter tilsidste et kort op, der viser hhv. ejendommen og forureningen.

In [ ]:
ejendomStyle = {'fillColor': '#228B22', 'color': '#228B22'}
dkjordStyle = {'color': '#FF0000'}

m = folium.Map(
    location=[56.1759, 10.6016],
    tiles="cartodbpositron",
    zoom_start=7,
)
ejendomLayer = folium.GeoJson(data=ejendom.geojson, name="ejendom", style_function=lambda x:ejendomStyle).add_to(m)
folium.GeoJson(data=dkjord.geojson, name="dkjord", style_function=lambda x:dkjordStyle).add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds(ejendomLayer.get_bounds())
m